In [1]:
import os
import csv
import numpy as np
import tensorflow as tf
from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense
from itertools import product
from keras_flops import get_flops
print("TensorFlow version:", tf.__version__)



def VGG19(input_shape):
    
    """
    Implementation of the popular ResNet50 the following architecture:
    CONV2D -> CONV2D -> MAXPOOL -> CONV2D -> CONV2D -> MAXPOOL -> CONV2D -> CONV2D -> CONV2D -> CONV2D -> MAXPOOL->
    CONV2D -> CONV2D -> CONV2D -> CONV2D -> MAXPOOL -> CONV2D -> CONV2D -> CONV2D -> CONV2D -> MAXPOOL ->
    CONV2D -> CONV2D -> CONV2D -> CONV2D -> MAXPOOL -> Flatten -> Dense -> Dense -> TopLayer
    

    Arguments:
    input_shape -- shape of the images of the dataset
    
    Returns:
    model -- a Model() instance in Keras
    """
    
    X_input = Input(input_shape)
    #Block 1
    X = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv1')(X_input)
    X = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv2')(X)
    X = MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool')(X)
    #Block 2
    X = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv1')(X)
    X = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv2')(X)
    X = MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool')(X)
    #Block 3
    X = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv1')(X)
    X = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv2')(X)
    X = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv3')(X)
    X = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv4')(X)
    X = MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool')(X)
    #Block 4
    X = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv1')(X)
    X = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv2')(X)
    X = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv3')(X)
    X = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv4')(X)    
    X = MaxPooling2D((2, 2), strides=(2, 2), name='block4_pool')(X)
    #Block 5
    X = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv1')(X)
    X = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv2')(X)
    X = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv3')(X)
    X = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv4')(X)    
    X = MaxPooling2D((2, 2), strides=(2, 2), name='block5_pool')(X)
    
    
    X = Flatten(name='flatten')(X)
    X = Dense(4096, activation='relu', name='fc1')(X)
    X = Dense(4096, activation='relu', name='fc2')(X)    
    
    # 输出层
    outputs = Dense(10, activation='softmax')(X)
    
    model = Model(inputs=X_input, outputs=outputs)

    
    return model

model_num = 1
input_shape_range =range(32,224,32)

with open("para_count_vgg19.csv","w") as csvfile:
         writer = csv.writer(csvfile)
         writer.writerows([["model_num", "input_horizontal", "input_vertical", "FLOPs", "Total_params", "Memory"],])
          
for shape_x in input_shape_range:
    shape_y = shape_x

    input_shape = (shape_x, shape_y, 1)
    model = VGG19(input_shape)
    model.summary()
    # how to compute the memory allocated by the activations of a model
    batch = 1
    shapes_count = int(np.sum([np.prod(np.array([s if isinstance(s, int) 
    else 1 for s in l.output_shape])) 
    for l in model.layers]))
    memory = (shapes_count * 4 * batch)/10**6
    print(memory)

    Total_params = round(model.count_params()/10 ** 6,2)
    #FLOPs = round(get_flops(model, batch_size=1)/10 ** 9,2)
    with open("para_count_vgg19.csv","a+") as csvfile:
        writer = csv.writer(csvfile)
        writer.writerows([[model_num, shape_x, shape_y, FLOPs, Total_params, memory],])

        model_num = model_num + 1

2022-04-02 03:46:22.752601: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/xilinx/xrt/lib:/usr/lib:/usr/lib/x86_64-linux-gnu:/usr/local/lib:/opt/vitis_ai/conda/envs/vitis-ai-tensorflow/lib
2022-04-02 03:46:22.752633: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


TensorFlow version: 2.6.0
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 32, 1)]       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 32, 32, 64)        640       
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 32, 32, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 16, 16, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 16, 16, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 16, 16, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None,

2022-04-02 03:46:23.752032: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/xilinx/xrt/lib:/usr/lib:/usr/lib/x86_64-linux-gnu:/usr/local/lib:/opt/vitis_ai/conda/envs/vitis-ai-tensorflow/lib
2022-04-02 03:46:23.752063: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-04-02 03:46:23.752077: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:163] no NVIDIA GPU device is present: /dev/nvidia0 does not exist
2022-04-02 03:46:23.752238: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 64, 64, 1)]       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 64, 64, 64)        640       
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 64, 64, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 32, 32, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 32, 32, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 32, 32, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 16, 16, 128)       0   

Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         [(None, 160, 160, 1)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 160, 160, 64)      640       
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 160, 160, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 80, 80, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 80, 80, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 80, 80, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 40, 40, 128)       0   

In [1]:
import os
import csv
import numpy as np
import tensorflow as tf
import keras
from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense
from itertools import product
from keras_flops import get_flops

def get_total_number_of_neurons(model, include_output_layer):
    '''
    Args:
        model: Keras model
        include_output_layer: A boolean parameter. Whether or not to include output layer's neurons into the calculation

    Returns: number of neurons in the given model
    '''
    num_layers = len(model.layers)
    total_num_of_neurons = 0
    for layer_index in range(num_layers):
        layer = model.layers[layer_index]
        # since we multiply layer output dimensions, initial value is set to 1.
        num_neurons_in_layer = 1
        for i in range(1, len(layer.output.shape)):
            try:
                # when it is a valid layer to count neurons, an output dimension of the layer can be convertible to int.
                num_neurons_in_layer *= int(layer.output.shape[i])
            except Exception:
                # if the output dimension of layer cannot be convertible to int,
                # just pass that layer since it is not a valid layer to count neurons
                pass
        # if num_neurons_in_layer is not still 1, it means we have a valid layer to count neurons
        if not num_neurons_in_layer == 1:
            # when it is an output layer
            if layer_index == (num_layers - 1):
                if include_output_layer:
                    total_num_of_neurons += num_neurons_in_layer
            else:  # when it is not an output layer
                total_num_of_neurons += num_neurons_in_layer
    return total_num_of_neurons

model_num = 1
input_shape_range =range(32,512,2)

with open("para_count_vgg19.csv","w") as csvfile:
         writer = csv.writer(csvfile)
         writer.writerows([["model_num", "input_horizontal", "input_vertical", "step","number_of_layers", "FLOPs_G", "Total_params_M", "Memory_M", "total_num_of_neurons_M"],])
path = "../vgg19_h5" #文件夹目录
files= os.listdir(path) #得到文件夹下的所有文件名称

for shape_x in input_shape_range:
    shape_y = shape_x
    for step in range(1,11):
        file = "vgg19_{input_x}_{input_y}_{step}.h5".format(input_x=shape_x,input_y=shape_y,step=step)
        if file in files: #遍历文件夹
            model_name = str(file)
            if not os.path.isdir(file): #判断是否是文件夹，不是文件夹才打开
                model = keras.models.load_model("../vgg19_h5/{}".format(model_name))
                num_neurons = get_total_number_of_neurons(model= model, include_output_layer= True)
                num_neurons_M = num_neurons/10**6
                # how to compute the memory allocated by the activations of a model
                batch = 1
                shapes_count = int(np.sum([np.prod(np.array([s if isinstance(s, int) 
                else 1 for s in l.output_shape])) 
                for l in model.layers]))
                memory = (shapes_count * 4 * batch)/10**6
                print(memory)
                layers_length = len(model.layers)
                Total_params = round(model.count_params()/10 ** 6,2)
                FLOPs = round(get_flops(model, batch_size=1)/10 ** 9,2)
                with open("para_count_vgg19.csv","a+") as csvfile:
                    writer = csv.writer(csvfile)
                    writer.writerows([[model_num, shape_x, shape_y, step, layers_length, FLOPs, Total_params, memory, num_neurons_M],])

                    model_num = model_num + 1

2022-05-16 04:21:23.752787: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/xilinx/xrt/lib:/usr/lib:/usr/lib/x86_64-linux-gnu:/usr/local/lib:/opt/vitis_ai/conda/envs/vitis-ai-tensorflow/lib
2022-05-16 04:21:23.752803: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-05-16 04:21:24.887474: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/xilinx/xrt/lib:/usr/lib:/usr/lib/x86_64-linux-gnu:/usr/local/lib:/opt/vitis_ai/conda/envs/vitis-ai-tensorflow/lib
2022-05-16 04:21:24.887491: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR

1.241852


2022-05-16 04:21:25.533888: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
2022-05-16 04:21:25.533989: I tensorflow/core/grappler/clusters/single_machine.cc:357] Starting new session
2022-05-16 04:21:25.553789: I tensorflow/core/grappler/optimizers/meta_optimizer.cc:1137] Optimization results for grappler item: graph_to_optimize
  function_optimizer: function_optimizer did nothing. time = 0.008ms.
  function_optimizer: function_optimizer did nothing. time = 0.001ms.



Instructions for updating:
Use `tf.compat.v1.graph_util.tensor_shape_from_node_def_name`

=========================Options=============================
-max_depth                  10000
-min_bytes                  0
-min_peak_bytes             0
-min_residual_bytes         0
-min_output_bytes           0
-min_micros                 0
-min_accelerator_micros     0
-min_cpu_micros             0
-min_params                 0
-min_float_ops              1
-min_occurrence             0
-step                       -1
-order_by                   float_ops
-account_type_regexes       .*
-start_name_regexes         .*
-trim_name_regexes          
-show_name_regexes          .*
-hide_name_regexes          
-account_displayed_op_only  true
-select                     float_ops
-output                     stdout:

==================Model Analysis Report======================

Doc:
scope: The nodes in the model graph are organized by their names, which is hierarchical like filesystem.
flops: Number

2022-05-16 04:21:26.956110: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
2022-05-16 04:21:26.956186: I tensorflow/core/grappler/clusters/single_machine.cc:357] Starting new session
2022-05-16 04:21:26.957839: I tensorflow/core/grappler/optimizers/meta_optimizer.cc:1137] Optimization results for grappler item: graph_to_optimize
  function_optimizer: function_optimizer did nothing. time = 0.004ms.
  function_optimizer: function_optimizer did nothing. time = 0.001ms.



...


=========================Options=============================
-max_depth                  10000
-min_bytes                  0
-min_peak_bytes             0
-min_residual_bytes         0
-min_output_bytes           0
-min_micros                 0
-min_accelerator_micros     0
-min_cpu_micros             0
-min_params                 0
-min_float_ops              1
-min_occurrence             0
-step                       -1
-order_by                   float_ops
-account_type_regexes       .*
-start_name_regexes         .*
-trim_name_regexes          
-show_name_regexes          .*
-hide_name_regexes          
-account_displayed_op_only  true
-select                     float_ops
-output                     stdout:

==================Model Analysis Report======================

Doc:
scope: The nodes in the model graph are organized by their names, which is hierarchical like filesystem.
flops: Number of float operations. Note: Please read the implementation for the math behind it.

Profi

2022-05-16 04:24:30.131825: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
2022-05-16 04:24:30.131899: I tensorflow/core/grappler/clusters/single_machine.cc:357] Starting new session
2022-05-16 04:24:30.133749: I tensorflow/core/grappler/optimizers/meta_optimizer.cc:1137] Optimization results for grappler item: graph_to_optimize
  function_optimizer: function_optimizer did nothing. time = 0.004ms.
  function_optimizer: function_optimizer did nothing. time = 0.001ms.




=========================Options=============================
-max_depth                  10000
-min_bytes                  0
-min_peak_bytes             0
-min_residual_bytes         0
-min_output_bytes           0
-min_micros                 0
-min_accelerator_micros     0
-min_cpu_micros             0
-min_params                 0
-min_float_ops              1
-min_occurrence             0
-step                       -1
-order_by                   float_ops
-account_type_regexes       .*
-start_name_regexes         .*
-trim_name_regexes          
-show_name_regexes          .*
-hide_name_regexes          
-account_displayed_op_only  true
-select                     float_ops
-output                     stdout:

==================Model Analysis Report======================

Doc:
scope: The nodes in the model graph are organized by their names, which is hierarchical like filesystem.
flops: Number of float operations. Note: Please read the implementation for the math behind it.

Profi

2022-05-16 04:24:32.971157: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
2022-05-16 04:24:32.971247: I tensorflow/core/grappler/clusters/single_machine.cc:357] Starting new session
2022-05-16 04:24:32.973022: I tensorflow/core/grappler/optimizers/meta_optimizer.cc:1137] Optimization results for grappler item: graph_to_optimize
  function_optimizer: function_optimizer did nothing. time = 0.004ms.
  function_optimizer: function_optimizer did nothing. time = 0.001ms.




=========================Options=============================
-max_depth                  10000
-min_bytes                  0
-min_peak_bytes             0
-min_residual_bytes         0
-min_output_bytes           0
-min_micros                 0
-min_accelerator_micros     0
-min_cpu_micros             0
-min_params                 0
-min_float_ops              1
-min_occurrence             0
-step                       -1
-order_by                   float_ops
-account_type_regexes       .*
-start_name_regexes         .*
-trim_name_regexes          
-show_name_regexes          .*
-hide_name_regexes          
-account_displayed_op_only  true
-select                     float_ops
-output                     stdout:

==================Model Analysis Report======================

Doc:
scope: The nodes in the model graph are organized by their names, which is hierarchical like filesystem.
flops: Number of float operations. Note: Please read the implementation for the math behind it.

Profi